# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import numpy as np
import pandas as pd
import os
import glob 
import os

from datetime import datetime

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml-bankrupt3'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: ops_analytics-workspace
Azure region: westeurope
Subscription id: 73f88e6b-3a35-4612-b550-555157e7059f
Resource group: ops_analytics-rg-gb-dev


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

In the current 


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# NOTE: update the key to match the dataset name
found = False
key = "Bankruptcu-dataset"
description_text = "Predicting Bankruptcy capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url = 'https://raw.githubusercontent.com/NAMU2NI/Capstone_Project/main/POC_level_features.csv'
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/data/dataset_factory.py:374: UserWarning: Ambiguous datetime formats inferred for columns ['Bankruptcy month'] are resolved as "month-day". Desired format can be specified by `set_column_types`.
  .format(column_types_builder.ambiguous_date_columns))


,Payer,FLAG_BAD_DEBT,# POCs,Rent,Average Receivables,Average Overdue,Payment Term Days,Credit Limit,Std Dev AR,Max Credit Utilization,Avg Credit Utilization,fb_rating
count,6.078000e+03,6078.000000,6078.000000,6078.000000,6.078000e+03,6.078000e+03,6078.000000,6078.000000,6078.000000,6.078000e+03,6.078000e+03,6078.000000
mean,3.028378e+07,0.007075,1.304870,858.841474,6.673015e+04,-3.510930e+03,17.821323,6955.941413,0.178076,1.492075e+04,1.216466e+04,1.301346
std,1.222117e+05,0.083820,6.907748,3701.399187,4.144858e+06,2.156629e+05,6.569184,17368.214761,2.371362,5.493294e+05,5.470089e+05,2.023613
min,3.012384e+07,0.000000,1.000000,0.000000,0.000000e+00,-1.666037e+07,0.000000,0.000000,0.000000,-1.831733e+06,-1.902979e+06,0.000000
25%,3.016368e+07,0.000000,1.000000,0.000000,0.000000e+00,-6.555792e+01,14.000000,2500.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
50%,3.023606e+07,0.000000,1.000000,0.000000,8.261600e+02,0.000000e+00,14.000000,2500.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
75%,3.041267e+07,0.000000,1.000000,0.000000,4.285353e+03,0.000000e+00,28.000000,6300.000000,0.095900,3.246082e-01,1.903275e-01,4.100000
max,3.047306e+07,1.000000,502.000000,85181.940000,3.217080e+08,5.826703e+05,63.000000,655600.000000,145.109226,4.195733e+07,4.195733e+07,5.000000


In [5]:
df.head(2)

,Payer,Bankruptcy month,FLAG_BAD_DEBT,# POCs,Rent,Average Receivables,Average Overdue,Payment Term Days,Payment Behavior Status,Credit Limit,Std Dev AR,Max Credit Utilization,Avg Credit Utilization,fb_rating
0,30123845,2019-01-01,0,1,0.0,36267.91,-35159.53,30,EXCELLENT,10000.0,0.248854,-3.520062,-3.735576,0.0
1,30155541,2019-01-06,0,1,0.0,0.00,0.00,28,EXCELLENT,2500.0,0.000000,0.000000,0.000000,0.0


In [6]:
from Train_Model import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

x, y = clean_data(dataset)
y_dataframe = pd.DataFrame(y)

all_data = pd.concat([x,y_dataframe], axis = 1)

In [7]:
Train, Test = train_test_split(
       all_data , test_size=0.33, random_state=0)

In [8]:
Train = Train.drop(['Payer', 'Bankruptcy month'], axis=1)
Test = Test.drop(['Payer', 'Bankruptcy month'], axis=1)

In [9]:
Train.rename(columns = {'FLAG_BAD_DEBT':'Y'}, inplace = True)
Test.rename(columns = {'FLAG_BAD_DEBT':'Y'}, inplace = True)

In [10]:
print('Shape Train: '+str(Train.shape))
print('Shape Test: '+str(Test.shape))

Shape Train: (4072, 15)
Shape Test: (2006, 15)


In [11]:
print ("# Bad Debts in training data",Train['Y'].sum())
print ("# Bad Debts in test data",Test['Y'].sum())

# Bad Debts in training data 28
# Bad Debts in test data 15


In [12]:
Train

,# POCs,Rent,Average Receivables,Average Overdue,Payment Term Days,Credit Limit,Std Dev AR,Max Credit Utilization,Avg Credit Utilization,fb_rating,Payment Behavior Status_AVERAGE,Payment Behavior Status_EXCELLENT,Payment Behavior Status_GOOD,Payment Behavior Status_STRUCTURAL OFFENDER,Y
6019,1,0.00,3454.070000,2783.691667,14,0.01,0.000000,247445.000000,207445.000000,0.0,0,0,0,1,0
1115,1,0.00,0.000000,0.000000,28,2500.00,0.000000,0.000000,0.000000,0.0,0,1,0,0,0
4899,1,0.00,0.000000,0.000000,14,3000.00,0.000000,0.000000,0.000000,3.8,0,1,0,0,0
5999,1,17509.36,20871.773330,-1266.953333,14,11100.00,0.259872,0.771139,0.435101,0.0,1,0,0,0,0
3804,1,0.00,6239.128333,0.000000,14,5000.00,0.095197,0.581832,0.478905,4.4,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,1,0.00,3779.543333,-3785.533333,14,2500.00,0.000000,0.000000,0.000000,4.5,0,1,0,0,0
3264,1,0.00,8.470000,0.000000,14,2500.00,0.000000,0.000000,0.000000,0.0,0,1,0,0,0
1653,1,1610.38,3451.195000,-673.473333,14,2500.00,0.165090,0.537976,0.290388,5.0,0,1,0,0,0
2607,1,0.00,0.000000,0.000000,14,2500.00,0.000000,0.000000,0.000000,0.0,0,1,0,0,0


In [13]:
from azureml.core import Dataset
from azureml.core import Workspace


ws = Workspace.from_config()
Train.to_csv("Data/Train.csv", index = False)

path = "Data/Train.csv"

data_store = ws.get_default_datastore()

try:
    train_dataset = Dataset.get_by_name(ws, name="Train")
except: 
    data_store.upload(src_dir = "./Data", target_path = './Data', overwrite = True,  show_progress = True) 
        
# Create TabularDataset & register in workspace
poc_dataset = Dataset.Tabular.from_delimited_files([(data_store, path)])
poc_dataset = poc_dataset.register(
        ws, name="poc_dataset", create_new_version=True,
        description="Dataset for POC Bankrupt prediction")       
        
#data_store.upload
#train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("project_3/train_data.csv"))

Uploading an estimated of 4 files
Uploading ./Data/.amlignore
Uploaded ./Data/.amlignore, 1 files out of an estimated total of 4
Uploading ./Data/.amlignore.amltmp
Uploaded ./Data/.amlignore.amltmp, 2 files out of an estimated total of 4
Uploading ./Data/Train.csv
Uploaded ./Data/Train.csv, 3 files out of an estimated total of 4
Uploading ./Data/train_data.csv
Uploaded ./Data/train_data.csv, 4 files out of an estimated total of 4
Uploaded 4 files


## Choose the Compute to Calculate 

In [14]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException 

# Choose a name for your CPU cluster
amlcompute_cluster_name = "Demo-Cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print("Creating a new compute cluster...")
    compute_config =  AmlCompute.provisioning_configuration(vm_size='STANDARD_D2S_V3', max_nodes=4, min_nodes=0, vm_priority='lowpriority') 
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
    
#aml_compute.wait_for_completion(show_output=True)
aml_compute.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)
#compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)


# use get_status() to get a detailed status for the current cluster. 
print(aml_compute.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-12-04T08:02:45.300000+00:00', 'errors': None, 'creationTime': '2021-09-17T05:18:08.686648+00:00', 'modifiedTime': '2021-09-17T05:18:24.335748+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2S_V3'}


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here

automl_settings = {
    "experiment_timeout_minutes": 40,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'norm_macro_recall'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task = "classification",
                             training_data=poc_dataset,
                             label_column_name="Y",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [16]:
automl_run = experiment.submit(automl_config,show_output = True)

Submitting remote run.
No run_configuration provided, running on Demo-Cluster with default configuration
Running on remote compute: Demo-Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml-bankrupt2,AutoML_87c7adcd-0386-4519-a580-943d7e9cb165,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a fal

        24                                                  0:02:07          nan    0.6842
ERROR: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_format": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_parameters": {},
        "reference_code": null,
        "details_uri": null,
        "target": null,
        "details": [],
        "inner_error": {
            "additional_properties": {},
            "code": "ResourceExhausted",
            "inner_error": {
                "additional_properties": {},
                "code": "Timeout",
                "inner_error": {
                    "additional_properties": {},
                    "code": "ExperimentTimeoutForIterations",
        

In [17]:
# Submit your automl run
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

#Wait for completion
automl_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_87c7adcd-0386-4519-a580-943d7e9cb165',
 'target': 'Demo-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-12-11T04:41:00.203159Z',
 'endTimeUtc': '2021-12-11T05:34:27.081987Z',
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 40 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'Demo-Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"auto-ml-bankrupt2","subscription_id":"73f88e6b-3a35-4612-b550-555157e7059f","resource_group":"ops_analytics-rg-gb-dev","workspace_name":"ops_analytics-workspace","region":"westeurope","compute_target":"Demo-Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enab

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
# Retrieve the best Automl Model run 
best_model, fitted_model = automl_run.get_output()
print(best_model)

# Get the Best Model and display properties 
model_name = best_model.properties['model_name']
print("Best Model:",model_name)

best_model.get_properties()

Package:azureml-automl-runtime, training version:1.36.1, current version:1.33.0
Package:azureml-core, training version:1.36.0.post2, current version:1.33.0
Package:azureml-dataprep, training version:2.24.4, current version:2.20.1
Package:azureml-dataprep-rslex, training version:2.0.3, current version:1.18.0
Package:azureml-dataset-runtime, training version:1.36.0, current version:1.33.0
Package:azureml-defaults, training version:1.36.0, current version:1.33.0
Package:azureml-interpret, training version:1.36.0, current version:1.33.0
Package:azureml-mlflow, training version:1.36.0, current version:1.33.0
Package:azureml-pipeline-core, training version:1.36.0, current version:1.33.0
Package:azureml-responsibleai, training version:1.36.0, current version:1.33.0
Package:azureml-telemetry, training version:1.36.0, current version:1.33.0
Package:azureml-train-automl-client, training version:1.36.0, current version:1.33.0
Package:azureml-train-automl-runtime, training version:1.36.1, current 

Run(Experiment: auto-ml-bankrupt2,
Id: AutoML_87c7adcd-0386-4519-a580-943d7e9cb165_25,
Type: azureml.scriptrun,
Status: Completed)
Best Model: AutoML87c7adcd025


{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'norm_macro_recall\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'auto-ml-bankrupt2\',\'compute_target\':\'Demo-Cluster\',\'subscription_id\':\'73f88e6b-3a35-4612-b550-555157e7059f\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_87c7adcd-0386-4519-a580-943d7e9cb165_25","experiment_name":"auto-ml-bankrupt2","workspace_name":"ops_analytics-workspace","subscription_id":"73f88e6b-3a35-4612-b550-555157e7059f","resource_group_name":"ops_analytics-rg-gb-dev"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '25',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.C

In [19]:
# Save the best model
os.makedirs('outputs', exist_ok=True)
best_model.download_file('outputs/model.pkl', 'outputs/automlmodel.pkl')
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_model.download_file('automl_driver.py', 'outputs/automl_driver.py')

In [20]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ="outputs/automlmodel.pkl",
                        model_name = "automl_Bankrupt_3")

print(model.name)

Registering model automl_Bankrupt_3
automl_Bankrupt_3


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
%%writefile autoscore.py
import json
import numpy as np
import os
import pickle
import joblib
from azureml.core.model import Model
import pandas as pd

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automlmodel.pkl')
    model = joblib.load(model_path)

    
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        mypredict = model.predict(data)
        return mypredict.tolist()
    except Exception as ex:
        error = str(ex)
        return error    

Overwriting autoscore.py


In [23]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


#env = Environment.from_conda_specification(name='puruenv',file_path = 'env.yml')
env = Environment.get(workspace=ws, name="AzureML-AutoML")
inference_config = InferenceConfig(entry_script='./autoscore.py',
                                    environment=env)

### Model Deployment 

In [25]:
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights = True, auth_enabled=True)
service = Model.deploy(ws, "bankrupt-model-auto-v1", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-11 07:06:56+00:00 Creating Container Registry if not exists.
2021-12-11 07:06:56+00:00 Registering the environment.
2021-12-11 07:06:57+00:00 Use the existing image.
2021-12-11 07:06:57+00:00 Generating deployment configuration.
2021-12-11 07:06:58+00:00 Submitting deployment to compute.
2021-12-11 07:07:07+00:00 Checking the status of deployment bankrupt-model-auto-v1..
2021-12-11 07:11:45+00:00 Checking the status of inference endpoint bankrupt-model-auto-v1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
servicelogs = service.get_logs()
print(servicelogs)

2021-12-11T07:11:35,032998900+00:00 - iot-server/run 
2021-12-11T07:11:35,054191200+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-12-11T07:11:35,058882600+00:00 - rsyslog/run 
2021-12-11T07:11:35,139301900+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_84c85d362f11658b9008714e1aa4657b/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-12-11T07:11:35,587269000+00:00 - iot-server/finish 1 0
2021-12-11T07:11:35,589882400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (71)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 99
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-12-11 07:11:40,469 | root | INFO | Starting up app insights client
logging socket was 

In [27]:
print("scoring URI: " + service.scoring_uri)
print("Swagger URI: " + service.swagger_uri)
print("Authetication Key: " + service.get_keys()[0])

scoring URI: http://6fade21e-5d07-4f72-b41f-67c870cbf0a0.westeurope.azurecontainer.io/score
Swagger URI: http://6fade21e-5d07-4f72-b41f-67c870cbf0a0.westeurope.azurecontainer.io/swagger.json
Authetication Key: HWzLx8ZbGuVVp7imdob7fJtV0Qs2Ql6g


In [45]:
3*3

9

In [28]:
x_data, y_data = clean_data(dataset)
x_data = x_data.drop(['Payer', 'Bankruptcy month'], axis=1)

In [29]:
x_data[4887:4889]

,# POCs,Rent,Average Receivables,Average Overdue,Payment Term Days,Credit Limit,Std Dev AR,Max Credit Utilization,Avg Credit Utilization,fb_rating,Payment Behavior Status_AVERAGE,Payment Behavior Status_EXCELLENT,Payment Behavior Status_GOOD,Payment Behavior Status_STRUCTURAL OFFENDER
4887,1,0.00,12015.97,20026.62,14,5300.00,0.00,4.53,4.53,0.00,0,1,0,0
4888,1,0.00,5531.77,1857.29,14,25000.00,0.03,0.19,0.15,0.00,0,0,0,1


In [33]:
x_data.columns

Index(['# POCs', 'Rent', 'Average Receivables', 'Average Overdue',
       'Payment Term Days', 'Credit Limit', 'Std Dev AR',
       'Max Credit Utilization', 'Avg Credit Utilization', 'fb_rating',
       'Payment Behavior Status_AVERAGE', 'Payment Behavior Status_EXCELLENT',
       'Payment Behavior Status_GOOD',
       'Payment Behavior Status_STRUCTURAL OFFENDER'],
      dtype='object')

In [30]:
import json

input_payload = json.dumps({
    'data': x_data[4887:4889].to_dict(orient='records'),
    'method': 'predict'  # classification model
})

output = service.run(input_payload)

print(output)

[1, 0]


In [34]:
import requests
import json
scoringuri = service.scoring_uri
key = service.get_keys()[0]
data= { "data":
       [
           {
               '# POCs':2,
               'Rent': 0,
               'Average Receivables': 10000,
               'Average Overdue': 50000,
               'Payment Term Days': 14,
               'Credit Limit': 0.01,
               'Std Dev AR': 0,
               'Max Credit Utilization': 24000,
               'Avg Credit Utilization':12000 ,
               'fb_rating': 0,
               'Payment Behavior Status_AVERAGE':0 ,
               'Payment Behavior Status_EXCELLENT':0 ,
               'Payment Behavior Status_GOOD':0,
               'Payment Behavior Status_STRUCTURAL OFFENDER':1
               
               
               
               
           },
           {
               '# POCs':1,
               'Rent': 0,
               'Average Receivables': 10000,
               'Average Overdue': 500,
               'Payment Term Days': 180,
               'Credit Limit': 4000,
               'Std Dev AR': 0,
               'Max Credit Utilization': 2000,
               'Avg Credit Utilization':0 ,
               'fb_rating': 4.5,
               'Payment Behavior Status_AVERAGE':0 ,
               'Payment Behavior Status_EXCELLENT':1 ,
               'Payment Behavior Status_GOOD':0,
               'Payment Behavior Status_STRUCTURAL OFFENDER':0
           }
       ]
    }
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)
print(response.text)

[1, 0]


In [31]:
print(service)

AciWebservice(workspace=Workspace.create(name='ops_analytics-workspace', subscription_id='73f88e6b-3a35-4612-b550-555157e7059f', resource_group='ops_analytics-rg-gb-dev'), name=bankrupt-model-auto-v1, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://6fade21e-5d07-4f72-b41f-67c870cbf0a0.westeurope.azurecontainer.io/score, properties={}, created_by={'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': 'a3c5b47ae395dd1677bbfc1ce6d42b58ab484eb0', 'mlflow.source.git.commit': 'a3c5b47ae395dd1677bbfc1ce6d42b58ab484eb0', 'azureml.git.dirty': 'True', 'hasInferenceSchema': 'False', 'hasHttps': 'False'})


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.